# Flood Risk Areas

## Import Libraries


In [2]:
# import libraries
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import requests
import warnings
warnings.filterwarnings('ignore')

## Prepare Data

### Import

In [3]:
# Define the API endpoint
api_url = "https://environment.data.gov.uk/flood-monitoring/id/floodAreas?&_limit=5000"

# Query the API
response = requests.get(url = api_url)

# Check if the request was successful
if response.status_code == 200:
    # convert response to json format
    flood_data = response.json()
    # Extract relevant data
    flood_areas = flood_data.get('items', [])

    # Prepare the data
    flood_areas_list = []
    for area in flood_areas:
        flood_areas_list.append({
            'county': area.get('county'),
            'description': area.get('description'),
            'eaAreaName': area.get('eaAreaName'),
            'lat': area.get('lat'),
            'long': area.get('long'),
            'riverOrSea': area.get('riverOrSea'),
            'polygon': area.get('polygon')
        })

    # Convert to DataFrame
    flood_areas_df = pd.DataFrame(flood_areas_list)
else:
    print(f"Failed to retrieve data: {response.status_code}")

In [4]:
# Display the DataFrame
flood_areas_df.head()

,county,description,eaAreaName,lat,long,riverOrSea,polygon
0,Lincolnshire,"Areas near the River Witham, Sincil Dyke, Grea...",Lincs and Northants,53.22002,-0.55462,"River Witham, Sincil Dyke, Great Gowts Drain",http://environment.data.gov.uk/flood-monitorin...
1,Lincolnshire,"Witham, Sincil Dyke, Great Gowts Drain, Fossdy...",Lincs and Northants,53.21764,-0.55269,"River Witham, Sincil Dyke, Great Gowts Drain, Fos",http://environment.data.gov.uk/flood-monitorin...
2,Cumberland,Properties affected by surface water flooding ...,Cumbria and Lancashire,54.60153,-3.13183,River Greta,http://environment.data.gov.uk/flood-monitorin...
3,"Cumberland, Westmorland and Furness",River Derwent from Keswick to Bassenthwaite. T...,Cumbria and Lancashire,54.61737,-3.14762,"River Greta, St Johns Beck, Glendermackin",http://environment.data.gov.uk/flood-monitorin...
4,Lancashire,"Properties on Church Street, Keirby Walk, Bank...",Cumbria and Lancashire,53.79078,-2.24018,River Brun,http://environment.data.gov.uk/flood-monitorin...


In [5]:
# Display information about the dataframe
flood_areas_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4182 entries, 0 to 4181
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   county       4182 non-null   object 
 1   description  4182 non-null   object 
 2   eaAreaName   4182 non-null   object 
 3   lat          4182 non-null   float64
 4   long         4182 non-null   float64
 5   riverOrSea   4130 non-null   object 
 6   polygon      4182 non-null   object 
dtypes: float64(2), object(5)
memory usage: 228.8+ KB


In [6]:
# Display dataframe dimension
flood_areas_df.shape

(4182, 7)

In [13]:
# new column names
column_names = ['county', 'text_description', 'area_name', 'lat', 'lon', 'water_source', 'polygon']
# change column names
flood_areas_df.columns = column_names

In [14]:
# Filter for areas near Merseyside
flood_areas_df = flood_areas_df[(flood_areas_df['area_name'].str.contains(
    'mersey', case=False, na=False))]

In [15]:
# Function to fetch and parse GeoJSON from the polygon URI
def fetch_geojson(uri):
    response = requests.get(uri)
    if response.status_code == 200:
        geojson = response.json()
        # Access the 'geometry' from the first feature
        if 'features' in geojson and len(geojson['features']) > 0:
            geometry = geojson['features'][0]['geometry']
            return shape(geometry)  # Convert GeoJSON to Shapely geometry
        else:
            print(f"No features found in GeoJSON data from {uri}")
            return None
    else:
        print(f"Failed to retrieve GeoJSON data from {uri}")
        return None


# Apply the function to the 'polygon' column
flood_areas_df['geometry'] = flood_areas_df['polygon'].apply(
    fetch_geojson)

In [16]:
# Convert the DataFrame to a GeoDataFrame
flood_areas_gdf = gpd.GeoDataFrame(flood_areas_df, geometry='geometry')

# Display the cleaned GeoDataFrame
flood_areas_gdf.head()

,county,text_description,area_name,lat,lon,water_source,polygon,geometry
28,Manchester,Land adjacent to the River Mersey at West Dids...,Gtr Mancs Mersey and Ches,53.41100,-2.24160,River Mersey,http://environment.data.gov.uk/flood-monitorin...,"POLYGON ((-2.24472 53.4153, -2.24508 53.41532,..."
29,"Cheshire East, Manchester, Salford, Stockport,...",The Middle River Mersey catchment includes Mic...,Gtr Mancs Mersey and Ches,53.43479,-2.31497,River Mersey,http://environment.data.gov.uk/flood-monitorin...,"MULTIPOLYGON (((-2.10956 53.32586, -2.10994 53..."
51,Manchester,Areas at risk include land and properties arou...,Gtr Mancs Mersey and Ches,53.51869,-2.22357,River Irk,http://environment.data.gov.uk/flood-monitorin...,"MULTIPOLYGON (((-2.22293 53.51522, -2.22231 53..."
52,"Bolton, Bury, Manchester, Oldham, Rochdale, Sa...",The Lower River Irwell catchment also includes...,Gtr Mancs Mersey and Ches,53.48905,-2.28848,River Irwell,http://environment.data.gov.uk/flood-monitorin...,"MULTIPOLYGON (((-2.23183 53.46984, -2.23233 53..."
64,"Manchester, Stockport","Areas in the locality of Mauldeth Road, includ...",Gtr Mancs Mersey and Ches,53.43947,-2.23322,Cringle Brook,http://environment.data.gov.uk/flood-monitorin...,"MULTIPOLYGON (((-2.22034 53.43457, -2.22038 53..."


## Export

In [79]:
# Save the DataFrame to a CSV file
flood_areas_df.to_csv('clean_data/flood_areas_data.csv', index=False)
print("Flood areas data saved to 'flood_areas_data.csv'")

Flood areas data saved to 'flood_areas_data.csv'


In [82]:
# Save the cleaned data to a new file (optional)
flood_areas_gdf.to_file('clean_data/flood_areas_data/cleaned_flood_areas_data.shp')